In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import sklearn
import math
import seaborn as sb
from IPython.display import display, Math, Latex
from matplotlib.legend_handler import HandlerLine2D
from collections import Counter
%matplotlib inline

Прежде, чем выбирать и тренировать модели, я решил поискать, что известно по этой теме:
<br>https://www.ruor.uottawa.ca/bitstream/10393/31553/3/Weissbock_Joshua_2014_thesis.pdf
<br>http://web.uvic.ca/~afyshe/dm_projs/nhl_final_report.pdf
<br>http://thesportjournal.org/article/goal-based-metrics-better-than-shot-based-metrics-at-predicting-hockey-success/

В целом, как я понял, модели основанные на подсчёте голов работают намного лучше, чем модели, основанные на подсчёте бросков по воротам.
<br>В файле очень стрёмные данные, возможно, я их обрабатываю не совсем эффективно, но мне ещё не приходилось тратить столько время на предобработку до обучения моделей :)

In [3]:
data2013_2014 = pd.read_csv("data2013-2014.csv")

/home/rtriangle/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data2013_2014.describe()

/home/rtriangle/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Unnamed: 0,Duration,Game.ID,Seconds,Home.Skaters,Away.Skaters,Home.Score,Away.Score,Score.Cat,Event.Length,...,fweight1,sweight1,gweight1,cweight2,fweight2,sweight2,gweight2,is.Rebound,is.Rush,xG
count,2.526505e+06,2.108858e+06,2.526447e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,...,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,2.526505e+06,112062.000000
mean,1.263253e+06,5.402609e+01,2.136209e+04,1.824997e+03,4.794811e+00,4.795009e+00,1.338297e+00,1.193619e+00,1.339206e-01,1.921711e+00,...,1.053428e+00,1.048019e+00,1.048531e+00,1.007583e+00,1.005990e+00,1.005763e+00,1.002349e+00,2.442109e-03,2.843850e-03,0.063799
std,7.293393e+05,9.878570e+01,2.592268e+03,1.080966e+03,7.438128e-01,7.241270e-01,1.369353e+00,1.286519e+00,1.374058e+00,5.043676e+00,...,2.665875e-01,2.468885e-01,2.424413e-01,8.744871e-02,7.730224e-02,7.578931e-02,4.758263e-02,4.935733e-02,5.325188e-02,0.066184
min,1.000000e+00,-1.200000e+03,2.000100e+04,-1.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.000000e+00,-1.000000e-03,...,6.247371e-01,6.328879e-01,6.514085e-01,8.370780e-01,8.505145e-01,8.488780e-01,9.337780e-01,0.000000e+00,0.000000e+00,0.003730
25%,6.316270e+05,NaN,NaN,8.910010e+02,5.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,...,9.012834e-01,9.046361e-01,9.273743e-01,9.616500e-01,9.602225e-01,9.635553e-01,9.453376e-01,0.000000e+00,0.000000e+00,NaN
50%,1.263253e+06,NaN,NaN,1.815001e+03,5.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,NaN
75%,1.894879e+06,NaN,NaN,2.745999e+03,5.000000e+00,5.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,9.000000e-03,...,1.123001e+00,1.117839e+00,1.084967e+00,1.041536e+00,1.043215e+00,1.039310e+00,1.061372e+00,0.000000e+00,0.000000e+00,NaN
max,2.526505e+06,1.200000e+03,3.041500e+04,6.026000e+03,6.000000e+00,6.000000e+00,9.000000e+00,8.000000e+00,3.000000e+00,8.099000e+01,...,2.504215e+00,2.381285e+00,2.151163e+00,1.241668e+00,1.213237e+00,1.216583e+00,1.076332e+00,1.000000e+00,1.000000e+00,0.535407


In [5]:
data2013_2014.columns

Index([u'Unnamed: 0', u'Period', u'Event', u'Description', u'Duration',
       u'Date', u'Game.ID', u'ev.team', u'p1', u'p2', u'p3', u'ev.zone',
       u'Detail', u'Seconds', u'a1.num', u'a1.pos', u'a2.num', u'a2.pos',
       u'a3.num', u'a3.pos', u'a4.num', u'a4.pos', u'a5.num', u'a5.pos',
       u'a6.num', u'a6.pos', u'h1.num', u'h1.pos', u'h2.num', u'h2.pos',
       u'h3.num', u'h3.pos', u'h4.num', u'h4.pos', u'h5.num', u'h5.pos',
       u'h6.num', u'h6.pos', u'Away.Team', u'Home.Team', u'Away.Goalie',
       u'Home.Goalie', u'Home.Skaters', u'Away.Skaters', u'Home.Score',
       u'Away.Score', u'Strength.State', u'Score.State', u'Score.Cat',
       u'Event.Length', u'URL', u'Highlight', u'XC', u'YC', u'Season',
       u'Season.Type', u'Home.Zone', u'Newcode', u'Round.Seconds', u'Distance',
       u'Angle', u'cweight1', u'fweight1', u'sweight1', u'gweight1',
       u'cweight2', u'fweight2', u'sweight2', u'gweight2', u'is.Rebound',
       u'is.Rush', u'xG'],
      dtype='object')

Я решил выделить признаки, несущие наибольшее количество информации для моей задачи, с помощью них можно будет сгенерировать также много новых фич.

In [6]:
data2013_2014_useful = data2013_2014[["Period", "Game.ID", "Away.Team", "Home.Team", 
                                      "Home.Score", "Away.Score", "Season", "Season.Type", 
                                      "Strength.State", "Score.State"]]

In [7]:
print data2013_2014_useful.shape
print Counter(np.array(data2013_2014_useful["Season"]))

(2526505, 10)
Counter({20132014: 2526505})


In [8]:
print data2013_2014_useful.shape
print Counter(np.array(data2013_2014_useful["Season.Type"]))

(2526505, 10)
Counter({'Regular': 2330144, 'Playoffs': 196361})


Таким образом, можем удалить колонку "Season", а из "Season.Type" сделать в будущем новый признак isRegular и isPlayOff

In [9]:
print Counter(data2013_2014_useful["Period"])

Counter({'1': 572493, '3': 556499, '2': 532299, 1: 284185, 3: 270162, 2: 263560, 'OT': 28376, 4: 7995, '4': 6900, '5': 2544, 5: 1460, 6: 30, nan: 2})


Правила об овертаймах в нхл меняются чуть ли не каждый год, но, в любом случае, давайте считать, что 1, 2, 3 - это основные периоды, а всё остальное - это 4ый период, то есть, овертайм.

In [10]:
di = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 4, '6': 4, 'OT': 4, 5: 4, 6: 4, np.nan: 4}

In [11]:
data2013_2014_useful = data2013_2014_useful.replace({"Period": di})

In [12]:
print Counter(data2013_2014_useful["Period"])
print Counter(data2013_2014_useful["Season"])

Counter({1: 856678, 3: 826661, 2: 795859, 4: 47307})
Counter({20132014: 2526505})


Cезон тоже можно удалить, потому что он не несёт никакой информации

In [13]:
del data2013_2014_useful["Season"]

Также видим, что Home.Score и Away.Score содержат ту же информацию, что и Score.State, поэтому удалим Score.State

In [14]:
del data2013_2014_useful["Score.State"]

Добавим новые признаки isRegular и isPlayoff

In [15]:
data2013_2014_useful['isRegular'] = np.where(data2013_2014_useful['Season.Type']=='Regular', 1, 0)
data2013_2014_useful['isPlayoff'] = np.where(data2013_2014_useful['Season.Type']!='Regular', 1, 0)
del data2013_2014_useful['Season.Type']

In [16]:
data2013_2014_useful.head()

,Period,Game.ID,Away.Team,Home.Team,Home.Score,Away.Score,Strength.State,isRegular,isPlayoff
0,1,20001.0,TOR,MTL,0,0,5v5,1,0
1,1,20001.0,TOR,MTL,0,0,5v5,1,0
2,1,20001.0,TOR,MTL,0,0,5v5,1,0
3,1,20001.0,TOR,MTL,0,0,5v5,1,0
4,1,20001.0,TOR,MTL,0,0,5v5,1,0


In [17]:
x = np.array(data2013_2014_useful["Strength.State"])
x1 = map(lambda y: (y.split('v')[0]), x)
x2 = map(lambda y: (y.split('v')[1]), x)

In [18]:
print Counter(x1)
print Counter(x2)

Counter({'5': 2166283, '4': 275918, 'E': 74032, '3': 6099, '0': 4055, '2': 63, '1': 55})
Counter({'5': 2151834, '4': 289256, 'E': 74032, '3': 7693, '0': 3651, '1': 24, '2': 15})


Я не знаю, что значит E в Strength.State, возможно, это equal, но в первых строчках видно, что в случае равных составов всё равно пишут XvX. Этих данных не так уж много по сравнению с числами, поэтому в будущем можем выкинуть строчки, где есть E

In [19]:
x1 = map(lambda x: float(x) if x != 'E' else 'E', x1)
x2 = map(lambda x: float(x) if x != 'E' else 'E', x2)

In [20]:
print Counter(x1)
print Counter(x2)

Counter({5.0: 2166283, 4.0: 275918, 'E': 74032, 3.0: 6099, 0.0: 4055, 2.0: 63, 1.0: 55})
Counter({5.0: 2151834, 4.0: 289256, 'E': 74032, 3.0: 7693, 0.0: 3651, 1.0: 24, 2.0: 15})


In [21]:
data2013_2014_useful["Away.Team.Strength"] = list(x1)
data2013_2014_useful["Home.Team.Strength"] = list(x2)

In [22]:
del data2013_2014_useful["Strength.State"]

In [23]:
data2013_2014_useful.shape

(2526505, 10)

In [24]:
data2013_2014_useful = data2013_2014_useful[data2013_2014_useful["Away.Team.Strength"] != 'E']
data2013_2014_useful = data2013_2014_useful[data2013_2014_useful["Home.Team.Strength"] != 'E']

In [25]:
data2013_2014_useful.shape

(2452473, 10)

Добавим информацию об удалениях, чтобы потом можно было считать количество удалений в периоде/матче и тд

In [26]:
x1 = list(data2013_2014_useful["Away.Team.Strength"])
x2 = list(data2013_2014_useful["Home.Team.Strength"])

In [27]:
x1 = map(lambda x: float(x), x1)
x2 = map(lambda x: float(x), x2)

In [28]:
x1_penalty = [float(max(x1[i] - x1[i+1], 0)) for i in range(len(x1)-1)] 
x2_penalty = [float(max(x2[i] - x2[i+1], 0)) for i in range(len(x2)-1)]
print np.sum(x1_penalty)
print np.sum(x2_penalty)

17497.0
18427.0


In [29]:
len(x1_penalty)

2452472

In [30]:
x1_penalty = x1_penalty + [0]
x2_penalty = x2_penalty + [0]

Так как уменьшили размерность на 1

In [31]:
data2013_2014_useful["Away.Penalty"] = np.array(x1_penalty)
data2013_2014_useful["Home.Penalty"] = np.array(x2_penalty)

Посмотрим на названия всех команд:

In [32]:
teams = list(set(np.array(data2013_2014_useful["Away.Team"])))
print teams

['N.J', 'MIN', 'TOR', 'WSH', 'BOS', 'DET', 'NYI', 'FLA', 'L.A', 'COL', 'NSH', 'DAL', 'CGY', 'NYR', 'CAR', 'WPG', 'BUF', 'VAN', 'STL', 'CHI', 'T.B', 'S.J', 'MTL', 'PHI', 'ANA', 'CBJ', 'PIT', 'EDM', 'ARI', 'OTT']


In [33]:
data2013_2014_useful.head()

,Period,Game.ID,Away.Team,Home.Team,Home.Score,Away.Score,isRegular,isPlayoff,Away.Team.Strength,Home.Team.Strength,Away.Penalty,Home.Penalty
0,1,20001.0,TOR,MTL,0,0,1,0,5,5,0.0,0.0
1,1,20001.0,TOR,MTL,0,0,1,0,5,5,0.0,0.0
2,1,20001.0,TOR,MTL,0,0,1,0,5,5,0.0,0.0
3,1,20001.0,TOR,MTL,0,0,1,0,5,5,0.0,0.0
4,1,20001.0,TOR,MTL,0,0,1,0,5,5,0.0,0.0


In [34]:
data2013_2014_useful.shape

(2452473, 12)

Я заметил, что есть nan в Game.ID, тяжело будет работать с такими данными, удалим их

In [35]:
data2013_2014_useful = data2013_2014_useful[np.isfinite(data2013_2014_useful["Game.ID"])]

In [36]:
data2013_2014_useful.shape

(2452416, 12)

In [37]:
keys_of_data_Home = data2013_2014_useful.groupby(['Home.Team', 'Game.ID']).groups.keys()
keys_of_data_Away = data2013_2014_useful.groupby(['Away.Team', 'Game.ID']).groups.keys()

In [38]:
keys_of_data_Home.sort()
keys_of_data_Away.sort()

In [39]:
gamesid_for_team = dict()
for team in teams:
    gamesid_for_team[team] = set()

In [40]:
for line in keys_of_data_Home:
    gamesid_for_team[line[0]].add(line[1])
for line in keys_of_data_Away:
    gamesid_for_team[line[0]].add(line[1])

Получили полезный набор Games.ID для каждой команды

In [41]:
print gamesid_for_team

{'N.J': set([20352.0, 20867.0, 20484.0, 20357.0, 21126.0, 20999.0, 20232.0, 20140.0, 20747.0, 20492.0, 20546.0, 20366.0, 20879.0, 20101.0, 20241.0, 20035.0, 20116.0, 20718.0, 20886.0, 21129.0, 21167.0, 21061.0, 20256.0, 20769.0, 21026.0, 20131.0, 21157.0, 20903.0, 20008.0, 20649.0, 20396.0, 20784.0, 20016.0, 20913.0, 21042.0, 20661.0, 20534.0, 20407.0, 21177.0, 20063.0, 20285.0, 21197.0, 20801.0, 20043.0, 20419.0, 20677.0, 21113.0, 20296.0, 20599.0, 21068.0, 20265.0, 20687.0, 20432.0, 20389.0, 20813.0, 20179.0, 20700.0, 20622.0, 20822.0, 20325.0, 21208.0, 20569.0, 21007.0, 20444.0, 20957.0, 21221.0, 20155.0, 20580.0, 21093.0, 20561.0, 20840.0, 20203.0, 20460.0, 20077.0, 20334.0, 20925.0, 20977.0, 20214.0, 20521.0, 20628.0, 20733.0, 20949.0]), 'MIN': set([20994.0, 20483.0, 20497.0, 21010.0, 30231.0, 30232.0, 30233.0, 30234.0, 30235.0, 30236.0, 21022.0, 21168.0, 21027.0, 20518.0, 20011.0, 20526.0, 21042.0, 20029.0, 21057.0, 20551.0, 20042.0, 21070.0, 20560.0, 20054.0, 20573.0, 21089.0, 2

In [42]:
keys_of_data_Games = data2013_2014_useful.groupby(['Game.ID']).groups.keys()

In [43]:
keys_of_data_Games.sort()

In [44]:
keys_of_data_Games

[20001.0,
 20002.0,
 20003.0,
 20004.0,
 20005.0,
 20006.0,
 20007.0,
 20008.0,
 20009.0,
 20010.0,
 20011.0,
 20012.0,
 20013.0,
 20014.0,
 20015.0,
 20016.0,
 20017.0,
 20018.0,
 20019.0,
 20020.0,
 20021.0,
 20022.0,
 20023.0,
 20024.0,
 20025.0,
 20026.0,
 20027.0,
 20028.0,
 20029.0,
 20030.0,
 20031.0,
 20032.0,
 20033.0,
 20034.0,
 20035.0,
 20036.0,
 20037.0,
 20038.0,
 20039.0,
 20040.0,
 20041.0,
 20042.0,
 20043.0,
 20044.0,
 20045.0,
 20046.0,
 20047.0,
 20048.0,
 20049.0,
 20050.0,
 20051.0,
 20052.0,
 20053.0,
 20054.0,
 20055.0,
 20056.0,
 20057.0,
 20058.0,
 20059.0,
 20060.0,
 20061.0,
 20062.0,
 20063.0,
 20064.0,
 20065.0,
 20066.0,
 20067.0,
 20068.0,
 20069.0,
 20070.0,
 20071.0,
 20072.0,
 20073.0,
 20074.0,
 20075.0,
 20076.0,
 20077.0,
 20078.0,
 20079.0,
 20080.0,
 20081.0,
 20082.0,
 20083.0,
 20084.0,
 20085.0,
 20086.0,
 20087.0,
 20088.0,
 20089.0,
 20090.0,
 20091.0,
 20092.0,
 20093.0,
 20094.0,
 20095.0,
 20096.0,
 20097.0,
 20098.0,
 20099.0,
 20100.0,


In [45]:
data2013_2014_useful_grouped_Games = data2013_2014_useful.groupby(['Game.ID']).max()
data2013_2014_useful_grouped_Games_sum = data2013_2014_useful.groupby(['Game.ID']).sum()
data2013_2014_useful_grouped_Games_mean = data2013_2014_useful.groupby(['Game.ID']).mean()

In [46]:
data2013_2014_useful_grouped_Games["Away.Penalty"] = data2013_2014_useful_grouped_Games_sum["Away.Penalty"]
data2013_2014_useful_grouped_Games["Home.Penalty"] = data2013_2014_useful_grouped_Games_sum["Home.Penalty"]

In [49]:
data2013_2014_useful_grouped_Games.head()

,Period,Away.Team,Home.Team,Home.Score,Away.Score,isRegular,isPlayoff,Away.Team.Strength,Home.Team.Strength,Away.Penalty,Home.Penalty
Game.ID,,,,,,,,,,,
20001.0,3,TOR,MTL,3,4,1,0,5.0,5.0,27.0,23.0
20002.0,3,WSH,CHI,6,4,1,0,5.0,5.0,16.0,18.0
20003.0,3,WPG,EDM,4,5,1,0,5.0,5.0,19.0,14.0
20004.0,3,TOR,PHI,1,3,1,0,5.0,5.0,17.0,23.0
20005.0,3,BUF,DET,2,1,1,0,5.0,5.0,29.0,9.0


### Вот тут мне бы хотелось иметь столбцы Away.Team.Strength и Home.Team.Strength, усреднённые по игре, но их нет(

In [50]:
data2013_2014_useful_grouped_Games_mean.head()

,Period,Home.Score,Away.Score,isRegular,isPlayoff,Away.Penalty,Home.Penalty
Game.ID,,,,,,,
20001.0,1.928302,1.538005,1.876550,1.0,0.0,0.014555,0.012399
20002.0,1.953065,2.431806,1.819437,1.0,0.0,0.008835,0.009939
20003.0,1.943135,2.741291,2.187500,1.0,0.0,0.009734,0.007172
20004.0,1.978355,0.653139,0.701299,1.0,0.0,0.009199,0.012446
20005.0,1.979708,1.570704,0.121750,1.0,0.0,0.018389,0.005707


In [93]:
del data2013_2014_useful_grouped_Games["Away.Team.Strength"]
del data2013_2014_useful_grouped_Games["Home.Team.Strength"]

In [96]:
print data2013_2014_useful_grouped_Games.shape

(1323, 9)
